# CD_3P1

### CD 3 Model, P1 Parameters

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16


### Load the Dataset

In [2]:
from tensorflow.keras.utils import image_dataset_from_directory

In [3]:
# Define paths to training and validation datasets
data_dir = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\DatasetConv'

# Load Datasets
train_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='training',     # Use the 'training' subset
    seed=123
)

val_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='validation',   # Use the 'validation' subset
    seed=123
)

Found 24997 files belonging to 2 classes.
Using 19998 files for training.
Found 24997 files belonging to 2 classes.
Using 4999 files for validation.


### Preprocessing & Augmentation

In [4]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [5]:
# Augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),  # 10% random rotation
    layers.RandomZoom(0.1),      # 10% zoom
    layers.RandomTranslation(0.1, 0.1),  # Random height and width shift
    layers.RandomBrightness(0.2)
])

# Augment the training data
def augment_img(image, label):
    image = data_augmentation(image)  # Apply augmentations
    return image, label

train_dataset = train_dataset.map(augment_img)

In [6]:
# Apply VGG-16 preprocessing
def preprocess_img(image, label):
    image = preprocess_input(image)  # Apply VGG16-specific preprocessing
    return image, label

train_dataset = train_dataset.map(preprocess_img)
val_dataset = val_dataset.map(preprocess_img)

### Load & Freeze the Pre-trained Model

In [7]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [8]:
# Freeze the layers of VGG16 so we don't retrain them
for layer in vgg_base.layers:
    layer.trainable = False

### Add the Head in the New Model & Compile

In [10]:
model = models.Sequential([
    vgg_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
# Define the checkpoint path
filepath = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\SavedModels\\'
model_name = 'CD3_P1'
checkpoint_path = f"{filepath}{model_name}_head_epoch{{epoch:03d}}_val{{val_loss:.4f}}.keras"

from tensorflow.keras.callbacks import ModelCheckpoint

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',              # Save every epoch
    save_weights_only=False,
    save_best_only=False,           # Save every time, not just best
    monitor='val_loss',
    verbose=1,
)

from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

### Train the Model

In [12]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback]
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9207 - loss: 0.1945
Epoch 1: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD3_P1_head_epoch001_val0.0485.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3164s 5s/step - accuracy: 0.9207 - loss: 0.1944 - val_accuracy: 0.9840 - val_loss: 0.0485
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9559 - loss: 0.1117
Epoch 2: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD3_P1_head_epoch002_val0.0474.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3012s 5s/step - accuracy: 0.9559 - loss: 0.1117 - val_accuracy: 0.9862 - val_loss: 0.0474
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9618 - loss: 0.0961
Epoch 3: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD3_P1_head_epoch003_val0.0471.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 2970

### Save History in JSON

In [14]:
import json

filepath = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\SavedModels\\'
model_name = 'CD3_P1'
# Assuming 'history' is the keras.callbacks.History object
filepath = f"{filepath}{model_name}_training_history.json"
with open(filepath, 'w') as f:
    json.dump(history.history, f)